In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import shap

# Sample dataset generation with continuous risk score
data = {
    'Payment_Amount': np.random.randint(5000, 100000, 100),
    'Delivery_Time': np.random.randint(10, 60, 100),
    'Contract_Size': np.random.choice(['Small', 'Large'], 100),
    'Contract_Duration': np.random.randint(100, 500, 100),
    'Buyer_Size': np.random.choice(['Small', 'Medium', 'Large'], 100),
    'Farmer_Experience': np.random.randint(1, 40, 100),
    'Buyer_Experience': np.random.randint(1, 40, 100),
    'Past_Contracts': np.random.randint(1, 50, 100),
    'Payment_Delays': np.random.randint(0, 10, 100),
    'Delivery_Delays': np.random.randint(0, 10, 100),
    'Disputes': np.random.randint(0, 5, 100),
    'Reviews': np.random.randint(1, 5, 100),
    'Weather_Conditions': np.random.choice(['Good', 'Moderate', 'Bad'], 100),
    'Market_Trends': np.random.choice(['Stable', 'Volatile'], 100),
    'Risk': np.random.randint(0, 101, 100)  # Continuous Risk Score from 0 to 100
}

df = pd.DataFrame(data)


In [16]:
from flask import Flask, render_template, request, jsonify
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

app = Flask(__name__)

# Assume you've already trained your XGBoost model
# Load the model

xgb_model = xgb.Booster()
xgb_model.load_model('xgb_model.json')  # Assuming the model is saved in a JSON format

# Initialize LabelEncoders (same as you used before)
contract_sizes = ['Small', 'Medium', 'Large']
weather_conditions = ['Low', 'Moderate', 'High']
market_trends = ['Stable', 'Growing', 'Declining']

le_contract_size = LabelEncoder().fit(contract_sizes)
le_weather_conditions = LabelEncoder().fit(weather_conditions)
le_market_trends = LabelEncoder().fit(market_trends)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    # Get form data from the HTML form
    payment_amount = int(request.form['payment_amount'])
    delivery_time = int(request.form['delivery_time'])
    contract_size = request.form['contract_size']
    contract_duration = int(request.form['contract_duration'])
    buyer_size = request.form['buyer_size']
    farmer_experience = int(request.form['farmer_experience'])
    buyer_experience = int(request.form['buyer_experience'])
    past_contracts = int(request.form['past_contracts'])
    payment_delays = int(request.form['payment_delays'])
    delivery_delays = int(request.form['delivery_delays'])
    disputes = int(request.form['disputes'])
    reviews = int(request.form['reviews'])
    weather_conditions_input = request.form['weather_conditions']
    market_trends_input = request.form['market_trends']
    
    # Prepare the data
    new_contract = {
        'Payment_Amount': payment_amount,
        'Delivery_Time': delivery_time,
        'Contract_Size': le_contract_size.transform([contract_size])[0],
        'Contract_Duration': contract_duration,
        'Buyer_Size': le_contract_size.transform([buyer_size])[0],
        'Farmer_Experience': farmer_experience,
        'Buyer_Experience': buyer_experience,
        'Past_Contracts': past_contracts,
        'Payment_Delays': payment_delays,
        'Delivery_Delays': delivery_delays,
        'Disputes': disputes,
        'Reviews': reviews,
        'Weather_Conditions': le_weather_conditions.transform([weather_conditions_input])[0],
        'Market_Trends': le_market_trends.transform([market_trends_input])[0]
    }

    # Convert the new contract data to a DataFrame
    new_contract_df = pd.DataFrame([new_contract])

    # Make a prediction using the model
    dmatrix = xgb.DMatrix(new_contract_df)
    predicted_risk_score = xgb_model.predict(dmatrix)

    # Return the prediction as a JSON response
    return jsonify({'risk_score': f"{predicted_risk_score[0]:.2f}"})

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\Ansh\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
